In [36]:
#important libraries
import pandas as pd
import numpy as np
import nltk
import re
import string
#importing stopwords is optional, in this case it decreased accuracy
import itertools
import time
from tqdm import tqdm
from langdetect import detect
from nltk.corpus import stopwords

#python imports
from sys import stdin,stdout
import re
import json
import os
from collections import Counter

#For visualization
from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from wordcloud import WordCloud,STOPWORDS
from tqdm import tqdm_notebook

In [37]:
data = pd.read_csv('text_emotion.csv')

In [38]:
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [39]:
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

In [40]:
data['content'] = data['content'].apply(lambda x: re.sub(r"https\S+", "", str(x)))
data['content'] = data['content'].apply(lambda x: x.replace("'", ""))
data['content'] = data['content'].apply(lambda x: x.translate(str.maketrans(string.punctuation, " "*len(string.punctuation))))
data['content'] = data['content'].apply(lambda x: x.lower())
data['content'] = data['content'].apply(lambda x: nltk.tokenize.word_tokenize(x))
data['content'] = data['content'].apply(lambda x: " ".join([lem.lemmatize(word, "v") for word in x]))
data['content'] = data['content'].apply(lambda x: x.strip())
data['content'].replace('', np.nan, inplace=True)
data.dropna(subset=['content'], inplace=True)

In [41]:
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,tiffanylue i know i be listenin to bad habit e...
1,1956967666,sadness,wannamama,layin n bed with a headache ughhhh waitin on y...
2,1956967696,sadness,coolfunky,funeral ceremony gloomy friday
3,1956967789,enthusiasm,czareaquino,want to hang out with friends soon
4,1956968416,neutral,xkilljoyx,dannycastillo we want to trade with someone wh...


In [42]:
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split

In [43]:
x_train, x_test, y_train, y_test = train_test_split(data.content, data.sentiment, test_size=0.25, random_state=0)
x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [73]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge',penalty='l2',alpha=1e-3,max_iter=10000,tol=1e-6,random_state=42))])

In [74]:
text_clf= text_clf_svm.fit(x_train,y_train)
y_pred = text_clf.predict(x_test)

In [75]:
np.mean(y_pred==y_test)

0.3214

In [50]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        27
     boredom       0.00      0.00      0.00        47
       empty       0.02      0.01      0.01       188
  enthusiasm       0.04      0.01      0.02       184
         fun       0.11      0.06      0.07       431
   happiness       0.33      0.31      0.32      1312
        hate       0.23      0.23      0.23       302
        love       0.39      0.53      0.45       976
     neutral       0.36      0.42      0.39      2176
      relief       0.12      0.07      0.09       367
     sadness       0.31      0.30      0.31      1326
    surprise       0.11      0.04      0.06       544
       worry       0.34      0.41      0.37      2120

    accuracy                           0.33     10000
   macro avg       0.18      0.18      0.18     10000
weighted avg       0.30      0.33      0.31     10000



In [76]:
vectorizer = TfidfVectorizer(min_df=3, max_df=0.9)

train_vectors = vectorizer.fit_transform(x_train)
test_vectors = vectorizer.transform(x_test)

model = svm.SVC(kernel='linear') 
model.fit(train_vectors, y_train) 
y_pred = model.predict(test_vectors)

In [77]:
np.mean(y_pred==y_test)

0.3508